# Defining chemical systems

Before performing chemical reaction calculations, we need to define the phases that must be considered in the calculation as well as the chemical species that constitute these phases.

**Example I:** to calculate the *solubility of a mineral in water*, two phases must be considered:
* *an aqueous phase* (representing water and dissolved species) and
* *a solid phase* (representing the mineral we want to dissolve).

**Example II:** to calculate the *combustion of a solid substance*, the chemical system must be composed of:
* *a gaseous phase* (with most of the gases relevant to the process) and
* *one or more solid/liquid substances* (condensed as phases) that may appear/disappear during the combustion process.

The concept of a chemical system (containing the information about phases) is implemented using the [ChemicalSystem](https://reaktoro.org/api/classReaktoro_1_1ChemicalSystem.html) class. It contains:
* a *list of phases* using [Phase](https://reaktoro.org/api/classReaktoro_1_1Phase.html) class (containing list of species using [Species](https://reaktoro.org/api/classReaktoro_1_1Species.html) class) and
* an *activity model* assigned to each phase of the list (to account for non-ideal thermodynamic behaviour).

## Chemical system for a mineral solubility problem

Let's consider the problem of determining the halite (NaCl) solubility in water. In this case, the [ChemicalSystem](https://reaktoro.org/api/classReaktoro_1_1ChemicalSystem.html) should be created with [AqueousPhase](https://reaktoro.org/api/classReaktoro_1_1AqueousPhase.html) and [MineralPhase](https://reaktoro.org/api/classReaktoro_1_1MineralPhase.html) classes.

In [ ]:
from reaktoro import *

# Define the PHREEQC database
db = PhreeqcDatabase("phreeqc.dat")

# Define an aqueous solution with species relevant to the problem
solution = AqueousPhase("H2O H+ OH- Na+ Cl-")

# Define a phase of the mineral which solubility is studied
halite = MineralPhase("Halite")

# Create the chemical system with the database and phases
system = ChemicalSystem(db, solution, halite)

Once the [ChemicalSystem](https://reaktoro.org/api/classReaktoro_1_1ChemicalSystem.html) instance is created, we can inspect its phases and chemical species composing them:

In [ ]:
for phase in system.phases():
    print(phase.name())
    for species in phase.species():
        print(":: " + species.name())

> **Note**: `AqueousPhase` and `GaseousPhase` are default names for aqueous and gaseous phases. For pure mineral phases, the phase name is the same as the name of the mineral species composing it.

> **Note**: Reaktoro supports as many phases as you wish, each phase containing any number of species.

We can inspect chemical species and elements of the system:

In [ ]:
print("Species :", end=" ")
for species in system.species():
    print(species.name(), end=" ")
print("\nElements:")
for element in system.elements():
    print(element.symbol())

The [ChemicalSystem](https://reaktoro.org/api/classReaktoro_1_1ChemicalSystem.html) class also contains the *formula matrix* $A$ of the system, which is a matrix whose entry $A_{j,i}$ contains the number of atoms of an element with index $j$ in species with index $i$:

In [ ]:
print(system.formulaMatrix())

The matrix printed above is the matrix of the size *5 x 6* corresponding to 5 components (4 elements and 1 charge) and 6 species we have inspected above. The last row in the formula matrix contains the electric charge of each species. For instance, H<sub>2</sub>0 corresponds to the first column with coefficient 2 for element H, 1 for element 0, and 0 for a charge.

## Chemical system for a gas solubility problem

Let us consider the chemical system for computation of the CO<sub>2</sub> solubility in saline solutions. Thus, we need an aqueous and gaseous phase (created by the [GaseousPhase](https://reaktoro.org/api/classReaktoro_1_1GaseousPhase.html) class):

In [ ]:
from reaktoro import *

# Define the SUPCRT database
db = SupcrtDatabase("supcrtbl")

# Define an aqueous solution with automatic species collection for selected elements
solution = AqueousPhase(speciate("H O Na Cl Ca C")) # HINT: 1) try to add N 2) try to add `exclude("organic")`
# HINT: 3) aqueous_species = [ "H2O(aq)", "CaOH+",  "CO2(aq)", "CO3-2", "CaCO3(aq)", "Ca+2", "Cl-", "H+", "HCO3-", "Na+", "OH-" ]

# Let Reaktoro automatically identify the gases by specifying an empty list of species below
gas = GaseousPhase() # HINT: try gas = GaseousPhase("CO2(g)")

# Create the chemical system
system = ChemicalSystem(db, solution, gas)

Below, we print the system's species, their names, formula, and molar mass:

In [ ]:
print(f"{len(system.species())} species found!")
print("{:<25}{:<25}{:<20}".format("Name", "Formula", "Molar Mass (kg/mol)"))
for species in system.species():
    print("{:<25}{:<25}{:<20.6f}".format(species.name(), species.formula().str(), species.molarMass()))

This chemical system contains many species due to the large amount of organic species included in the SUPCRTBL database. To filter them out, we can use the `exclude()` function:

~~~python
solution = AqueousPhase(speciate("H O Na Cl Ca C"), exclude("organic"))
~~~

> **Warning:** the specified species' names must coincide exactly the names they are included in [Database](https://reaktoro.org/api/classReaktoro_1_1Database.html).

As earlier, we can specify the exact aqueous species to be considered in the phase:

In [ ]:
aqueous_species = [
    "H2O(aq)",
    "CaOH+", 
    "CO2(aq)",
    "CO3-2", 
    "CaCO3(aq)",
    "Ca+2", 
    "Cl-",
    "H+", 
    "HCO3-", 
    "Na+",
    "OH-"
]

solution = AqueousPhase(aqueous_species)

gas = GaseousPhase() 

system = ChemicalSystem(db, solution, gas)

for species in system.species():
    print(species.name())

## Chemical system with many phases

Now, let's define a chemical system with many phases and species using `cemdata18` database provided by ThermoFun, which is suitable for modeling cement chemistry. We define *an aqueous solution* with the chemical elements of interest. Since all minerals could potentially be important (e.g., minerals that could precipitate as the solution temperature is changed), we let Reaktoro to collect *all minerals* in the database whose constituting elements are found in the aqueous solution. Moreover, we show how the mineral phase can be defined as a *solid solution* (containing more than one mineral end-member).

In [ ]:
# Define the ThermoFun database
db = ThermoFunDatabase("cemdata18")

# Define phases
solution = AqueousPhase(speciate("H O Na Cl Ca Mg C Si Fe Al K S"))
gas = GaseousPhase()
pureminerals = MineralPhases()
solidsolution = MineralPhase("ettringite Fe-ettringite").setName("SolidSolution")

# Create the chemical system
system = ChemicalSystem(db, solution, gas, pureminerals, solidsolution)

for phase in system.phases():
    print(phase.name())
    for species in phase.species():
        print(":: " + species.name())

**TASK 1**: 1) define a chemical system with two minerals (albite and kaolinite) and aqueous solution (allowing Reaktoro to automatically generate its content) 2) inspect, which aqueous species were selected automatically for the aqueous phase.

In [ ]:
# -------------------------------------- #
# Code cell for the task
# -------------------------------------- #

## Chemical system for a combustion problem

Let's construct a chemical system suitable for modeling the *combustion of black powder*. Black powder is composed of potassium nitrate (KNO<sub>3</sub>), charcoal (C<sub>10</sub>Ca<sub>0.026</sub>H<sub>4.774</sub>N<sub>0.039</sub>O<sub>1.234</sub>), and sulfur (S<sub>8</sub>). In the code below, we construct a [CondensedPhases](https://reaktoro.org/api/classReaktoro_1_1CondensedPhases.html) using the chemical elements above:

In [ ]:
# Define the NASA-CEA database
db = NasaDatabase("nasa-cea")

# Initialize condensed phases (solid or liquid substances) with given elements
condensed = CondensedPhases(speciate("K N O C Ca H S"))

# Automatically select the gases based on the elements above
gases = GaseousPhase()  

# Create a chemical system
system = ChemicalSystem(db, gases, condensed)

# Output gases and condensed phases constituting the chemical system
print("Gases")
for species in system.species():
    if species.aggregateState() == AggregateState.Gas:
        print(":: " + species.name())
print("Condensed Phases")
for species in system.species():
    if species.aggregateState() == AggregateState.CondensedPhase:
        print(":: " + species.name())

> **Note**: by including as many gases and condensed phases as possible, Reaktoro will be able to determine those that may appear (find positive amounts for them) after burning black powder. For example, K<sub>2</sub>S(cd), K<sub>2</sub>SO<sub>4</sub>(cd) and CaS(cd) are typically formed in the combustion process. By including them in the definition of the chemical system, the chemical equilibrium solver will be able to find positive amounts for them (i.e., the solver will identify these condensed phases as stable in equilibrium).


**TASK 2**: check how many phases and species are collected in the chemical system.

In [ ]:
# -------------------------------------- #
# Code cell for the task
# -------------------------------------- #